In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
RU_WEBS_DIR = "./pages-arch/pages-arch/ru"
EN_WEBS_DIR = "./pages-arch/pages-arch/en"

ru_webs_pl = glob(f"{RU_WEBS_DIR}/**.html")
en_webs_pl = glob(f"{EN_WEBS_DIR}/**.html")
print(f"Russian websites amount:{len(ru_webs_pl)}")
print(f"English websites amount:{len(en_webs_pl)}")

Russian websites amount:195
English websites amount:565


In [3]:
raw_ru = []
raw_en = []

for pl in ru_webs_pl:
    with open(pl, "r") as f:
        raw_ru.append(f.read())
for pl in en_webs_pl:
    with open(pl, "r") as f:
        raw_en.append(f.read())     

In [4]:
ru_df = pd.DataFrame({"raw": raw_ru})
ru_df["lang"] = "RU"
en_df = pd.DataFrame({"raw": raw_en})
en_df["lang"] = "EN"
df = pd.concat([ru_df, en_df])
df.head()

,raw,lang
0,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU
1,"<!doctype html><html class=""hcfe"" data-page-ty...",RU
2,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU
3,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU
4,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU


In [ ]:
#!pip install html2text

In [5]:
import html2text
h2t = html2text.HTML2Text()
h2t.ignore_links = True
df['text'] = df['raw'].apply(h2t.handle)

In [30]:
#!python -m spacy download en_core_web_sm
#!python -m spacy download ru_core_news_sm

/home/atheostheos/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-02-28 21:06:25.021400: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 21:06:25.058934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 21:06:25.058970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 21:06:25.059809: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 21:06:25.065363: I tensorflow/core/platform/cpu_f

2024-02-28 21:06:38.399471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-28 21:06:39.387633: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 4.8 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [6]:
import spacy
import spacy_transformers
nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")
en_df = df[df['lang'] == "EN"]
ru_df = df[df['lang'] == "RU"]

/home/atheostheos/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-02-29 16:28:39.175262: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 16:28:39.348868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 16:28:39.348932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 16:28:39.369440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 16:28:39.416913: I tensorflow/core/platform/cpu_f

In [7]:
def get_tokens_en(text):
    doc = nlp_en(ex)
    return [w for w in doc if (w.is_alpha and not w.is_stop)]

def get_tokens_ru(text):
    doc = nlp_ru(ex)
    return [w for w in doc if (w.is_alpha and not w.is_stop)]

In [100]:
ts = get_tokens_en(en_df.iloc[0]["text"])

In [118]:
ts[69].lemma_

'image'

In [71]:
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('omw-1.4')

def has_meaning(word):
    return len(wordnet.synsets(word)) > 0

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/atheostheos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/atheostheos/nltk_data...


In [76]:
#!pip install wiki-ru-wordnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Preparing metadata (setup.py) ... done
  Created wheel for wiki-ru-wordnet: filename=wiki_ru_wordnet-1.0.3-py3-none-any.whl size=20890429 sha256=87a9429ac8c523f0a5035a497a10bbd184896ef3d98a012a6886574c3d24764b
  Stored in directory: /home/atheostheos/.cache/pip/wheels/37/2d/4a/6ff7974a42f9f1adc93919dc2c107e3358abaec46580332d81
Successfully built wiki-ru-wordnet


In [80]:
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()

def has_meaning_ru(word):
    return len(wikiwordnet.get_synsets(word)) > 0

In [103]:
from tqdm.notebook import tqdm
import re

pattern = re.compile(r'^[А-Яа-яёЁ]+$|^[A-Za-z]+$')

def get_info(df, lang="EN"):
    token_set = set()
    lemmas_set = dict()
    for index, row in tqdm(df.iterrows()):
        lang = row['lang']
        doc = nlp_en(row['text']) if lang == "EN" else nlp_ru(row["text"])
        toks = [w for w in doc if (w.is_alpha and not w.is_stop)]
        #token_set.update([t.lower_ for t in toks])
        for t in toks:
            # filter text with invslid characters OR that's too short OR too long 
            if not pattern.match(t.text) or len(t.text) < 2 or len(t.text) > 29:
                continue
            # filter tokens that in vocabulary
            if (lang=="EN" and not has_meaning(t.lemma_)) or (lang=="RU" and not has_meaning_ru(t.lemma_)):
                continue
            lem = t.lemma_
            token_set.update([t.lower_])
            if lem in lemmas_set.keys():
                lemmas_set[lem].update([t.lower_])
            else:
                lemmas_set[lem] = set([t.lower_])
    
    return token_set, lemmas_set    

In [102]:
has_meaning_ru("nih")

False

In [104]:
en_token_set, en_lemmas_set = get_info(en_df, lang="EN")

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [105]:
ru_token_set, ru_lemmas_set = get_info(ru_df, lang="RU")

0it [00:00, ?it/s]

In [106]:
with open("ru_tokens.txt", "w") as f:
    for word in ru_token_set:
        f.write(word + "\n")
with open("en_tokens.txt", "w") as f:
    for word in en_token_set:
        f.write(word + "\n")

In [107]:
with open('ru_lemmas.txt', "w") as f:
    for key, lem_set in ru_lemmas_set.items():
        f.write(f"{key.lower()}: {' '.join(lem_set)}\n")
with open('en_lemmas.txt', "w") as f:
    for key, lem_set in en_lemmas_set.items():
        f.write(f"{key.lower()}: {' '.join(lem_set)}\n")